In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns 
import pickle as pkl
from Utils.data_preparation import get_feature_set

## Cross Validation Results with Multiple Feature Sets

### Plot average results in heatmap

In [ ]:
models = ['CoxPH', 'Reg_Cox', 'RSF', 'GBT']
evaluation_strategy = 'all_pooled' 

metrics_to_report = ['Harrell_C', 'IBS'] 
feature_sets = ['Multi', 'Cog_Demo', 'Image_Demo', 'Demo'] 
feature_set_names = {
    'Multi': 'All',
    'Cog_Demo': 'Cog_Demo',
    'Image_Demo': 'Image_Demo',
    'Demo': 'Demo'
}

imputation_method = 'Complete_Case' #['Complete_Case', 'KNN_imputer_no_outcome', 'KNN_imputer_with_outcome', 'MF_imputer_no_outcome', 'MF_imputer_with_outcome']

if evaluation_strategy == 'external_harmo_split':
    dataset_names = {
        'train_valid': 'Training',
        'test': 'Internal Validation',
        'harmo': 'HARMO',
        'harmo_high_SVD': 'HARMO High SVD',
        'harmo_low_SVD': 'HARMO Low SVD',
    }
elif evaluation_strategy == 'all_pooled':
    dataset_names = {
        'train_valid': 'Training',
        'test': 'Testing'#'Internal Validation'
    }

full_results = []
for model in models:
    for feature_set in feature_sets:
        this_dict_entry = {
            'Model': model,
            'Feature Set': feature_set_names[feature_set]
        }

        # Read in full validation results
        input_variables_to_print, FS_name, var_description, cat_feature_indices = get_feature_set(feature_set)
        folderpath = '/Users/lirui/Downloads/Cohort_Dementia_Prediction/Survival/Nested_CV_Results/{}/{}/{}/{}'.format(imputation_method, evaluation_strategy, model, FS_name)
        results_for_this_model_FS = pkl.load( open(folderpath+'/{}_{}.pkl'.format(model, FS_name), 'rb') )['full_validation_results']

        for dataset in list(dataset_names.keys()):
            for metric in metrics_to_report:
                for value in ['mean', 'std']:
                    column_name = '{}_{}_{}'.format(dataset, metric, value)
                    this_dict_entry[column_name] = results_for_this_model_FS[column_name].iloc[0]

                column_name = '{}_{}_values'.format(dataset, metric)
                this_dict_entry[column_name] = results_for_this_model_FS[column_name].iloc[0]

                column_name = '{}_{}_mean_std'.format(dataset, metric)
                this_dict_entry[column_name] = '{:.3f} ({:.3f})'.format(
                    this_dict_entry['{}_{}_mean'.format(dataset, metric)],
                    this_dict_entry['{}_{}_std'.format(dataset, metric)]
                )

        full_results.append(this_dict_entry)
full_results_df = pd.DataFrame(full_results)
full_results_df.head()


In [ ]:
sns.set(font_scale=1.2)
print_dataset={
    'train': 'Training',
    'test': 'Test'
}

dataset_to_report = 'test'
col_param = 'Feature Set'
column_order = [feature_set_names[fs] for fs in feature_sets]
idx_param = 'Model'
index_order = models

fig, axes = plt.subplots(1, len(metrics_to_report), figsize=(5.5*len(metrics_to_report),5))
for i, metric in enumerate(metrics_to_report):
    

    results_table = pd.pivot_table(full_results_df, values=dataset_to_report+'_'+metric+'_mean', columns=col_param, index=idx_param)
    results_table = results_table.reindex(index_order, axis=0)
    results_table = results_table.reindex(column_order, axis=1)

    if metric == 'IBS':
        axes[i] = sns.heatmap(results_table, ax=axes[i], annot=True, fmt='.3f', cmap='Greens') #sns.cm.rocket_r)
    else:
        axes[i] = sns.heatmap(results_table, ax=axes[i], annot=True, fmt='.3f', cmap='Greens_r') #sns.cm.rocket) 
    
    if metric == 'Harrell_C':
        axes[i].set_title('{} C-Index '.format(print_dataset[dataset_to_report]), fontsize=14, fontweight='bold')
    else:
        axes[i].set_title('{} {} '.format(print_dataset[dataset_to_report],metric), fontsize=14, fontweight='bold')

    
    axes[i].set_xlabel(col_param, fontsize=14, fontweight='bold')
    axes[i].set_ylabel(idx_param, fontsize=14, fontweight='bold')

    tl = axes[i].get_yticklabels()
    axes[i].set_yticklabels(tl, rotation=0)

    tl = axes[i].get_xticklabels()
    axes[i].set_xticklabels(tl, rotation=45)
#plt.yticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('/Users/lirui/Downloads/Cohort_Dementia_Prediction/Survival/Nested_CV_Results/Complete_Case/all_pooled/test_results.pdf')

In [ ]:
selected_dataset = 'test'
selected_metric = 'IBS'

col_param = 'Feature Set'
column_order = feature_sets
idx_param = 'Model'
index_order = models

results_table = pd.pivot_table(full_results_df, values=selected_dataset+'_'+selected_metric+'_mean_std', columns=col_param, index=idx_param, aggfunc=lambda x: ' '.join(str(v) for v in x))
results_table = results_table.reindex(index_order, axis=0)
results_table = results_table.reindex(column_order, axis=1)

print(selected_dataset, selected_metric)
results_table

### Test for significant difference between models

In [ ]:
from scipy.stats import ttest_rel

model_1 = 'CoxPH'
model_2 = 'GBT'

print(model_1, model_2)
for dataset in ['test']:
    for metric in metrics_to_report:
        for feature_set in feature_sets:
            
            row_for_model_1 = full_results_df.loc[(full_results_df['Model']==model_1) & (full_results_df['Feature Set']==feature_set)]
            
            row_for_model_2 = full_results_df.loc[(full_results_df['Model']==model_2) & (full_results_df['Feature Set']==feature_set)]

            column_name = '{}_{}_values'.format(dataset, metric)
            values_1 = row_for_model_1[column_name].iloc[0]
            values_2 = row_for_model_2[column_name].iloc[0]

            assert len(values_1) == len(values_2) == 5
            
            statistic, pval = ttest_rel(values_1, values_2)
            if pval <0.05: # statistic<0 indicates values_1 less than values_2
                if statistic<0:
                    print('{}\t{}\t{}\t pval:{}\t {} larger'.format(dataset, metric, feature_set, pval, model_2))
                else:
                    print('{}\t{}\t{}\t pval:{}\t {} larger'.format(dataset, metric, feature_set, pval, model_1))
            

### Extract individual results entry

In [ ]:
# sns_pallete = {
#     'Logistic': sns.color_palette()[0],
#     'Reg_Logistic': sns.color_palette()[2],
#     'GMLVQ': sns.color_palette()[1]}

models = ['CoxPH', 'Reg_Cox', 'RSF', 'GBT']
evaluation_strategy = 'all_pooled' # Choose among: external_harmo, external_harmo_split, RUN_DMC_SCANS_only, all_pooled

metrics_to_report = ['Harrell_C', 'IBS'] #['Harrell_C', 'Uno_C', 'IBS']
feature_sets = ['Multi', 'Cog_Demo', 'Image_Demo', 'Demo'] # 'Image_Demo_no_PSMD'
imputation_methods = ['Complete_Case'] #['Complete_Case', 'KNN_imputer_no_outcome', 'KNN_imputer_with_outcome', 'MF_imputer_no_outcome', 'MF_imputer_with_outcome']

if evaluation_strategy == 'external_harmo_split':
    dataset_names = {
        'train_valid': 'Training',
        'test': 'Internal Validation',
        'harmo': 'HARMO',
        'harmo_high_SVD': 'HARMO High SVD',
        'harmo_low_SVD': 'HARMO Low SVD',
    }
elif evaluation_strategy == 'all_pooled':
    dataset_names = {
        'train_valid': 'Training',
        'test': 'Testing'#'Internal Validation'
    }

model_FS_dataset_metric_imputation_df_lst = []
for imputation_method in imputation_methods:
    model_FS_dataset_metric_df_lst = []
    for model in models:
        FS_dataset_metric_df_lst = []
        for FS_idx, this_feature_set in enumerate(feature_sets):
            # Read in full validation results
            input_variables_to_print, FS_name, var_description, cat_feature_indices = get_feature_set(this_feature_set)
            folderpath = '/Users/lirui/Downloads/Cohort_Dementia_Prediction/Survival/Nested_CV_Results/{}/{}/{}/{}'.format(imputation_method, evaluation_strategy, model, FS_name)
            this_results_dict = pkl.load( open(folderpath+'/{}_{}.pkl'.format(model, FS_name), 'rb') )
            this_validation_results = this_results_dict['full_validation_results']

            dataset_metric_df_lst = []
            for dataset in list(dataset_names.keys()):
                dataset_metric_df = {}
                for metric in metrics_to_report:
                    # if metric == 'Harrell_C':
                    #     dataset_metric_df['C-Index'] = this_validation_results[dataset+'_'+metric+'_values'].iloc[0]
                    # else:
                    dataset_metric_df[metric] = this_validation_results[dataset+'_'+metric+'_values'].iloc[0]

                dataset_metric_df = pd.DataFrame.from_dict(dataset_metric_df)
                dataset_metric_df['Dataset'] = dataset_names[dataset]
                dataset_metric_df_lst.append(dataset_metric_df)
            
            FS_dataset_metric_df = pd.concat(dataset_metric_df_lst)
            FS_dataset_metric_df['Feature Set'] = this_feature_set
            FS_dataset_metric_df_lst.append(FS_dataset_metric_df)
        model_FS_dataset_metric_df = pd.concat(FS_dataset_metric_df_lst)
        model_FS_dataset_metric_df['Model'] = model
        model_FS_dataset_metric_df_lst.append(model_FS_dataset_metric_df)
    model_FS_dataset_metric_imputation_df = pd.concat(model_FS_dataset_metric_df_lst)
    model_FS_dataset_metric_imputation_df['Imputation'] = imputation_method
    model_FS_dataset_metric_imputation_df_lst.append(model_FS_dataset_metric_imputation_df)

complete_df = pd.concat(model_FS_dataset_metric_imputation_df_lst)
print(complete_df.shape)
complete_df.head()

### Option 1: Plot results

In [ ]:
sns.set(font_scale = 1.2)
metrics_to_plot = metrics_to_report
models_to_print = models
datasets = list(dataset_names.keys())
fig, axes = plt.subplots(len(metrics_to_plot), len(datasets), figsize=(5.8*len(datasets),4.7*len(metrics_to_plot)), sharex=False)
for i, metric in enumerate(metrics_to_plot):
    for j, dataset in enumerate(datasets):
        results_for_this_dataset = complete_df.loc[complete_df['Dataset']==dataset_names[dataset]]
        sns.boxplot(x="Feature Set", y=metric, hue="Model", hue_order=models_to_print, data=results_for_this_dataset, ax=axes[i,j], showfliers=False, width=0.5)# fliersize=2
        axes[i,j].set_title(dataset_names[dataset], fontweight='bold')
        # sns.pointplot(x="Feature Set", y=metric, hue="Model", hue_order=models_to_print, data=results_for_this_dataset, ax=axes[i,j], showfliers=False, join=True, errwidth=1)
        if i != len(datasets)-1:
            axes[i,j].set_xlabel('')
        else:
            axes[i,j].set_xlabel('Feature Set', fontweight='bold')
        
        if metric == 'Harrell_C':
            axes[i,j].set_ylabel('C-Index', fontweight='bold')
        else:
            axes[i,j].set_ylabel(metric, fontweight='bold')

        #if (i==1 and j==(len(datasets)-1))==False:
        axes[i,j].get_legend().remove()
        
        if metric=='IBS':
            axes[i,j].set_ylim([0.02,0.1])
        else:
            axes[i,j].set_ylim([0.6,1.0])
        #axes[i,j].set_title(dataset_names[dataset])
        plt.setp(axes[i,j].get_xticklabels(), rotation=36, ha="right",rotation_mode="anchor")
axes[0,len(datasets)-1].legend(bbox_to_anchor=(1.05, 1.05))
#fig.suptitle(np.unique(complete_df['Imputation']))
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('/Users/lirui/Downloads/Cohort_Dementia_Prediction/Survival/Nested_CV_Results/{}/{}/Survival_CV_Boxplots.png'.format(imputation_methods[0], evaluation_strategy), bbox_inches='tight')

### Option 2: Get summary statistics in a table

In [ ]:
metrics_to_report = metrics
models_to_report = models
feature_sets_to_report = feature_sets
imputation_methods_to_report = imputation_methods
assert len(imputation_methods)==1

imputation_dataset_feature_metric_model_results_df_lst = []
for imputation_method in imputation_methods_to_report:
    dataset_feature_metric_model_results_df_lst = []
    for dataset in [dataset_names[i] for i in list(dataset_names.keys())]:
        feature_metric_model_results_df_lst = []
        for feature_set in feature_sets_to_report:
            metric_model_results = {}
            for metric in metrics_to_report:
                metric_model_results[metric] = {}
                for model in models_to_report:
                    selected_df = complete_df.loc[(complete_df['Dataset']==dataset) & (complete_df['Feature Set']==feature_set) & (complete_df['Model']==model) & (complete_df['Imputation']==imputation_method)]
                    metric_model_results[metric][model] = {
                        'Mean': np.round(np.mean(selected_df[metric]),3),
                        'SD': np.round(np.std(selected_df[metric]),3)
                    }
            feature_metric_model_results_df = pd.DataFrame.from_dict({(i,j): metric_model_results[i][j] 
                                    for i in metric_model_results.keys() 
                                    for j in metric_model_results[i].keys()},
                                orient='columns')
            feature_metric_model_results_df = feature_metric_model_results_df.reset_index(drop=False, col_level=1).rename(columns={'index': 'value'})
            feature_metric_model_results_df.insert(0, 'Feature Set', [feature_set]*feature_metric_model_results_df.shape[0])
            feature_metric_model_results_df_lst.append(feature_metric_model_results_df)
        dataset_feature_metric_model_results_df = pd.concat(feature_metric_model_results_df_lst)
        dataset_feature_metric_model_results_df.insert(0, 'Dataset', dataset)
        dataset_feature_metric_model_results_df_lst.append(dataset_feature_metric_model_results_df)
    imputation_dataset_feature_metric_model_results_df = pd.concat(dataset_feature_metric_model_results_df_lst)
    imputation_dataset_feature_metric_model_results_df.insert(0, 'Imputation', imputation_method)
    imputation_dataset_feature_metric_model_results_df_lst.append(imputation_dataset_feature_metric_model_results_df)

complete_summary_results_df = pd.concat(imputation_dataset_feature_metric_model_results_df_lst)
print(complete_summary_results_df.shape)
complete_summary_results_df.head()

In [ ]:
complete_summary_results_df.to_csv('/Users/lirui/Downloads/Cohort_Dementia_Prediction/Survival/Nested_CV_Results/{}/{}/Results Summary.csv'.format(imputation_methods_to_report[0], evaluation_strategy), index=False)